Para este endpoint tengo que devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [2]:
df_reviews = pq.read_table('Datos/df_reviews_sentimiento.parquet').to_pandas()
df_steam_games = pq.read_table('Datos/df_steam_games_limpio_final.parquet').to_pandas()


Selecciono las columnas de los dataframes que voy a usar para el análisis

In [3]:
df_reviews = df_reviews[['user_id','recommend','item_id']]
df_steam_games = df_steam_games[['price','id']]

Chequeo que las columnas con las que voy a hacer el merge (nuevamente inner join) sean del mismo tipo.

In [4]:
type(df_reviews['item_id'][0]),type(df_steam_games['id'][0])

(numpy.int32, numpy.int64)

Hago el merge almacenandolo en la variable "resultado"

In [5]:
resultado = df_reviews.merge(df_steam_games, left_on='item_id', right_on='id', how='inner')

Dropeo la columna 'id', ya que no la necesito más

In [6]:
resultado.drop('id',axis=1,inplace=True)
resultado.shape

(17207410, 4)

Ya que 'resultado' es muy grande , creo un nuev df pero que sólo mantenga las combinaciones únicas de 'user_id' e 'item_id'

In [7]:
df = resultado.drop_duplicates(subset=['user_id', 'item_id'])


In [9]:
df.shape

(52511, 4)

In [14]:
df.head()

,user_id,recommend,item_id,price
0,76561197970982479,true,1250,19.99
1,death hunter,true,1250,19.99
2,djkamber,true,1250,19.99
3,diego9031,true,1250,19.99
4,76561198081962345,true,1250,19.99


In [11]:
df.reset_index(inplace=True)

In [13]:
df.drop(columns=['index'], axis=1, inplace=True)

C:\Users\diego\AppData\Local\Temp\ipykernel_7608\3666756689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['index'], axis=1, inplace=True)


In [ ]:
df['user_id'][0]

'76561197970982479'

In [15]:
import pyarrow.parquet as pq
df.to_parquet('df_endpoint3.parquet')

Creo dos variables nuevas: cantidad y dinero. Cantidad almacena un recuento de recommend:true y recommend:false, mientras que dinero cuenta la cantidad de dinero gastada por el usuario. Usaría el primer usuario que encuentro para probar que funcione todo, pero me parece mejor probar con alguno que tenga tantos comentarios positivos como negativos

In [20]:
cantidad = df[(df['user_id'] == 'mercoledi')].groupby('recommend').count()

dinero = df[(df['user_id'] == 'mercoledi')].groupby('price').count()

In [21]:
cantidad

,user_id,item_id,price
recommend,,,
false,2,2,2
true,3,3,3


In [24]:
dinero

,user_id,recommend,item_id
price,,,
0.0,1,1,1
14.99,1,1,1
19.99,2,2,2
9.99,1,1,1


spent es el dinero gastado introducido en una lista

In [25]:
spent = dinero.index.tolist()

In [26]:
recommend = cantidad.index.tolist()
cantidad_items_recom = cantidad['item_id'].tolist()

In [29]:
recommend, cantidad_items_recom, spent

(['false', 'true'], [2, 3], ['0.0', '14.99', '19.99', '9.99'])

In [30]:
spent = [float(value) for value in spent]

Estoy probando la cantidad_items_recom[1] (los recommend=True) / la suma de los items * 100 para sacar el porcentaje de recomendación sobre el total, y por otro lado la suma del dinero gastado por el ususario

In [31]:
cantidad_items_recom[1]/sum(cantidad_items_recom)*100, sum(cantidad_items_recom), sum(spent)

(60.0, 5, 44.97)

In [34]:
dicc = {"Usuario X" : 'mercoledi', "Dinero gastado": sum(spent), "% de recomendación": cantidad_items_recom[1]/sum(cantidad_items_recom)*100, "cantidad de items": sum(cantidad_items_recom)}

In [35]:
dicc

{'Usuario X': 'mercoledi',
 'Dinero gastado': 44.97,
 '% de recomendación': 60.0,
 'cantidad de items': 5}